In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ==================================================
# 0. 의존성
# ==================================================
import os, random, time
from pathlib import Path
from collections import Counter
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data     import Dataset, DataLoader, WeightedRandomSampler
from torchvision          import transforms, models

# --------------------------------------------------
ROOT         = "/content/drive/MyDrive/Data/MVTecAD/bottle"
TRAIN_DIR    = os.path.join(ROOT, "train")
TEST_DIR     = os.path.join(ROOT, "test")

IMG_SIZE     = # TODO
BATCH_SIZE   = # TODO
NUM_EPOCHS   = # TODO
VAL_RATIO    = 0.2
LR           = # TODO
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# ==================================================
# 1. 전역 클래스 매핑 + 전체 샘플 리스트
# ==================================================
def scan_classes(*dirs):
    return sorted({p.name for d in dirs for p in Path(d).iterdir() if p.is_dir()})

classes        = scan_classes(TRAIN_DIR, TEST_DIR)
class_to_idx   = {cls: i for i, cls in enumerate(classes)}
num_classes    = len(classes)
print("라벨:", classes)

def samples_from(root):
    return [(str(p), class_to_idx[p.parent.name])
            for p in Path(root).rglob("*.*") if p.is_file() and p.parent.name in class_to_idx]

all_samples = samples_from(TRAIN_DIR) + samples_from(TEST_DIR)
random.shuffle(all_samples)                       # 시드 고정했으므로 재현 가능
print(f"총 이미지: {len(all_samples)}")

라벨: ['broken_large', 'broken_small', 'contamination', 'good']
총 이미지: 292


In [ ]:
# ==================================================
# 2. 학습/검증 분할
# ==================================================
val_size   = int(len(all_samples) * VAL_RATIO)
val_samples, train_samples = all_samples[:val_size], all_samples[val_size:]

print(f"학습 {len(train_samples)}  |  검증 {len(val_samples)}")

학습 234  |  검증 58


In [ ]:
# ==================================================
# 3. 커스텀 데이터셋
# ==================================================

# ImageNet-1K (ILSVRC 2012)
mean, std = [0.485,0.456,0.406], [0.229,0.224,0.225]
train_tf  = transforms.Compose([
    transforms.Resize((#TODO, #TODO)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(#TODO),
    transforms.ToTensor(), transforms.Normalize(mean, std),
])
val_tf    = transforms.Compose([
    transforms.Resize((#TODO, #TODO)),
    transforms.ToTensor(), transforms.Normalize(mean, std),
])

class SimpleImageDS(Dataset):
    def __init__(self, samples, transform=None):
        self.samples, self.t = samples, transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        return (self.t(img) if self.t else img), label

train_set = SimpleImageDS(#TODO, #TODO)
val_set   = SimpleImageDS(#TODO, #TODO )

In [ ]:
# ==================================================
# 4. 클래스 가중치 & Weighted Sampler
# ==================================================
cnt          = Counter([lbl for _, lbl in train_samples])
total_train  = len(train_samples)
class_weights = torch.tensor(
    [ (total_train / cnt[i]) if cnt[i]>0 else 0.0 for i in range(num_classes)],
    dtype=torch.float, device=DEVICE)

sample_weights = [class_weights[lbl].item() for _, lbl in train_samples]
sampler = WeightedRandomSampler(#TODO, num_samples=#TODO, replacement=True)

train_loader = DataLoader(#TODO, batch_size=#TODO, sampler=#TODO,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(#TODO, batch_size=#TODO, shuffle=False,
                          num_workers=4, pin_memory=True)

In [ ]:
# ==================================================
# 5. 모델 (EfficientNet-B0)
# ==================================================
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
for p in model.parameters():         # 백본 고정
    p.requires_grad = False
model.classifier[1] = nn.Linear(model.classifier[1].in_features, #TODO)
model.to(DEVICE)

criterion = nn.CrossEntropyLoss(weight=#TODO)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=#TODO)


In [ ]:
# ==================================================
# 6. 학습 루프 (변경 없음)
# ==================================================
def run_epoch(loader, train=True):
    model.train() if train else model.eval()
    loss_sum = correct = total = 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        with torch.set_grad_enabled(train):
            out  = #TODO
            loss = #TODO
            if train:
                optimizer.zero_grad();
                loss.backward();
                optimizer.step()
        loss_sum += loss.item() * x.size(0)
        correct  += (out.argmax(1)==y).sum().item()
        total    += y.size(0)
    return loss_sum/total, correct/total

best_acc = 0
for e in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    tr_loss,tr_acc = run_epoch(train_loader,True)
    va_loss,va_acc = run_epoch(val_loader,  False)
    if va_acc > best_acc:
        best_acc = va_acc; torch.save(model.state_dict(),"best_effb0_cls.pth")
    print(f"[{e:02d}] Train {tr_loss:.4f}/{tr_acc:.3f} | "
          f"Val {va_loss:.4f}/{va_acc:.3f} | {time.time()-t0:.1f}s")

print("최고 검증 정확도:", best_acc )

[01] Train 1.1683/0.427 | Val 1.2690/0.138 | 3.4s
[02] Train 0.9037/0.526 | Val 1.1452/0.172 | 3.3s
[03] Train 0.7961/0.581 | Val 1.0530/0.190 | 3.4s
[04] Train 0.7727/0.594 | Val 0.9873/0.155 | 3.3s
[05] Train 0.7165/0.590 | Val 0.9231/0.190 | 3.3s
[06] Train 0.6212/0.662 | Val 0.9474/0.155 | 3.3s
[07] Train 0.5351/0.645 | Val 0.9060/0.155 | 3.3s
[08] Train 0.4842/0.650 | Val 0.9019/0.172 | 3.3s
[09] Train 0.5375/0.611 | Val 0.8741/0.172 | 3.3s
[10] Train 0.4849/0.632 | Val 0.8458/0.207 | 3.4s
[11] Train 0.4741/0.662 | Val 0.8095/0.224 | 3.4s
[12] Train 0.4518/0.641 | Val 0.7846/0.310 | 3.3s
[13] Train 0.4580/0.632 | Val 0.7704/0.379 | 3.3s
[14] Train 0.4390/0.675 | Val 0.7673/0.379 | 3.3s
[15] Train 0.4468/0.667 | Val 0.7356/0.552 | 3.4s
[16] Train 0.5278/0.645 | Val 0.7472/0.397 | 3.3s
[17] Train 0.3719/0.705 | Val 0.7458/0.379 | 3.3s
[18] Train 0.3463/0.748 | Val 0.6878/0.776 | 3.4s
[19] Train 0.3694/0.705 | Val 0.6530/0.776 | 3.3s
[20] Train 0.4171/0.654 | Val 0.6361/0.879 | 3.3s


In [ ]:
# ==================================================
# 7. Test Visualization  ── 검증(=test) 이미지 예측 결과 보기
# ==================================================
import matplotlib.pyplot as plt
import numpy as np

# ① 최고 성능 모델 불러오기
model.load_state_dict(torch.load("best_effb0_cls.pth", map_location=DEVICE))
model.eval()

# ② 시각화 함수
def unnormalize(img_tensor):
    """(C,H,W) 텐서를 0~1 범위 numpy 이미지로 복원"""
    mean = torch.tensor([0.485, 0.456, 0.406]).reshape(3,1,1)
    std  = torch.tensor([0.229, 0.224, 0.225]).reshape(3,1,1)
    img = img_tensor.cpu() * std + mean
    return img.permute(1,2,0).numpy().clip(0,1)

def show_predictions(loader, num_images=8):
    shown = 0
    rows  = int(np.ceil(num_images / 4))
    plt.figure(figsize=(16, 4 * rows))

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            probs = torch.softmax(model(imgs), dim=1)
            preds = probs.argmax(1).cpu()
            probs = probs.cpu()

            for i in range(imgs.size(0)):
                if shown >= num_images: break
                shown += 1

                plt.subplot(rows, 4, shown)
                plt.imshow(unnormalize(imgs[i]))
                gt   = classes[labels[i].item()]
                pred = classes[preds[i].item()]
                conf = probs[i, preds[i]].item()
                title = f"Pred: {pred} ({conf:.2f})\nGT: {gt}"
                plt.title(title, fontsize=9)
                plt.axis("off")
            if shown >= num_images: break
    plt.tight_layout()
    plt.show()

# ③ 호출
show_predictions(val_loader, num_images=32)   # ← 원하는 수로 조정


Output hidden; open in https://colab.research.google.com to view.